In [1]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import plotly.express as px
import matplotlib.pyplot as plt

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [2]:
df=pd.read_csv('translated_data_l5000.csv')
data=df.head(30)

In [3]:
data

,og_article,og_summary,og_headline,article,summary,headline
0,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,ब्लैकबेरी हैंडसेट बनाने वाली कनाडा की कंपनी रि...,रिम करेगा दो हजार कर्मचारियों की छंटनी,"The Canadian company Research in Motion Rim, w...","The Canadian company Research in Motion Rim, w...",Rim will be trimmed of two thousand employees
1,अमेरिका में राष्ट्रपति पद के लिए डेमोक्रेटिक प...,श्रद्धांजलि सभा के दौरान अचानक खराब हुई हिलेरी...,अमेरिका राष्ट्रपति पद की उम्मीदवार हिलेरी क्लि...,"Hillary Clinton, a Democratic Party candidate ...",Hillary Clinton's health that suddenly deterio...,US Presidential candidate Hillary Clinton conf...
2,इंडिगो एयरलाइन के प्रेसीडेंट और डायरेक्टर आदित...,इंडिगो के ग्राउंड स्टाफ ने यात्री से की हाथापा...,इंडिगो के स्टाफ ने दिल्ली एयरपोर्ट पर पैसेंजर ...,"Aditya Ghosh, President and director of Indigo...",Indigo's ground staff has apologized for this ...,Indigo staff assaulted passenger at Delhi Airp...
3,संजय लीला भंसाली की फिल्म मलाल से बॉलीवुड में ...,मीजान जाफरी ने किया खुलासानव्या नंदा को बताया ...,नव्या नवेली के साथ दोस्ती पर जावेद जाफरी के बे...,"Mejan Jafri, who made her Bollywood debut with...","Meijan Jafri opted out to Nanda, a good friend...",Javed Jaffrey's son revealed this on friendshi...
4,इस पूरे मामले की जांच उत्तर-प्रदेश सरकार द्वार...,समर्थकों के साथ पुलिस अधिकारी के आवास के बाहर...,उन्नाव रेप केस आधी रात को आरोपी विधायक कुलदीप ...,The SIT is being investigated by the Uttar Pra...,The supporters had reached outside the police ...,Unnao rape case did not surrender in front of ...
5,विवादित इस्लामिक प्रचारक ज़ाकिर नाइक के गैर सर...,ज़ाकिर नाइक के बैंक खातों की एनआईए जांच कर रही...,इस्लामिक धर्मगुरु ने रियल एस्टेट में किया करोड...,The Government of India is being tightened by ...,NIA is investigating Zakir Naik's bank account...,Islamic religious leader invests crores of rup...
6,राष्ट्रपति प्रतिभा पाटील रविवार सुबह सेशेल्स औ...,राष्ट्रपति प्रतिभा पाटील रविवार सुबह सेशेल्स औ...,राष्ट्रपति दिवसीय दौरे पर सेशेल्स व द अफ्रीका ...,President Pratibha Patil left for a nine-day v...,President Pratibha Patil left for a nine-day v...,Seychelles and The Africa leave for presidenti...
7,उन्होंने कहा कि अपने क्षेत्रों की समस्याओं को ...,कहा- जब संसद नहीं चल रही हो तो अपने क्षेत्र मे...,पीएम मोदी मिले बीजेपी के ओबीसी सांसदों से सरका...,He said that by meeting Union Ministers and of...,"Said- When the Parliament is not going on, the...",PM Modi met BJP's OBC MPs for strengthening th...
8,सलमान खान ने इस तस्वीर को ट्वीट कर लिखा जितने...,सलमान खान दिखे बुजुर्ग के गेटअप मेंउनकी तस्वीर...,सलमान खान दिखे बुजुर्ग के गेटअप में लिखा- जित...,Salman Khan tweeted this picture and wrote tha...,Salman Khan looked like his picture in the get...,Salman Khan was seen in the old man's getup- t...
9,रोम में सोमालियाई समुद्री लुटेरों ने इटली के ए...,फरवरी में इटली के इस जहाज का अपहरण किया गया था...,सोमालियाई लुटेरों ने भारतीयों को किया रिहा,Somalian pirates in Rome were released by an o...,"The Italian ship was kidnapped in February, in...",Somalian robbers released Indians


In [4]:
text = data['article']
goldsummary = data['summary']

In [5]:
bart_checkpoint = "facebook/bart-large"

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
tokenizer_bart = AutoTokenizer.from_pretrained(bart_checkpoint, 
                                               do_lower_case=False, 
                                               use_fast=False, 
                                               keep_accents=True)

model_bart = AutoModelForSeq2SeqLM.from_pretrained(bart_checkpoint).to(device)

In [8]:
bos_id = tokenizer_bart._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer_bart._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer_bart._convert_token_to_id_with_added_voc("<pad>")

In [9]:
%%time

SystemSummary = []

for i, input_text in enumerate(text):
    
    inp = tokenizer_bart(input_text, add_special_tokens=False, 
                         truncation=True, return_tensors="pt", 
                         padding='max_length', max_length=1024)['input_ids'].to(device)  
    
    model_output = model_bart.generate(inp, use_cache=True, 
                                        num_beams=4, 
                                        max_length=60, 
                                        min_length=20, 
                                        early_stopping=True, 
                                        pad_token_id=pad_id,
                                        bos_token_id=bos_id, 
                                        eos_token_id=eos_id, 
                                        decoder_start_token_id=tokenizer_bart._convert_token_to_id_with_added_voc("<2en>"))
    
    decoded_output = tokenizer_bart.decode(model_output[0], 
                                    skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=False)
    
    SystemSummary.append(decoded_output)

Wall time: 15min 19s


In [10]:
Summaries = pd.DataFrame(list(zip(text,goldsummary, SystemSummary)), columns =['Articles','GoldSummary', 'BartSummary'])
Summaries

,Articles,GoldSummary,BartSummary
0,"The Canadian company Research in Motion Rim, w...","The Canadian company Research in Motion Rim, w...",Preparing restructuring. Canadian company Rese...
1,"Hillary Clinton, a Democratic Party candidate ...",Hillary Clinton's health that suddenly deterio...,This news has not been edited by the NDTV team...
2,"Aditya Ghosh, President and director of Indigo...",Indigo's ground staff has apologized for this ...,"itya Ghosh, President and director of Indigo A..."
3,"Mejan Jafri, who made her Bollywood debut with...","Meijan Jafri opted out to Nanda, a good friend...","jan Jafri, who made her Bollywood debut with S..."
4,The SIT is being investigated by the Uttar Pra...,The supporters had reached outside the police ...,Uttar Pradesh SIT is being investigated by the...
5,The Government of India is being tightened by ...,NIA is investigating Zakir Naik's bank account...,Has has invested about crores of rupees in rea...
6,President Pratibha Patil left for a nine-day v...,President Pratibha Patil left for a nine-day v...,President Patil is with President Patil Seyche...
7,He said that by meeting Union Ministers and of...,"Said- When the Parliament is not going on, the...",Met the Lok Sabha and Rajya Sabha MPs of two o...
8,Salman Khan tweeted this picture and wrote tha...,Salman Khan looked like his picture in the get...,man Khan tweeted this picture and wrote that t...
9,Somalian pirates in Rome were released by an o...,"The Italian ship was kidnapped in February, in...",omalian pirates in Rome were released by an oi...


In [11]:
rouge = Rouge()
score = rouge.get_scores(Summaries['BartSummary'], Summaries['GoldSummary'], avg=True)
BartRouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
BartRouge


,rouge-1,rouge-2,rouge-l
recall,0.454576,0.243486,0.395305
precision,0.263440,0.125514,0.228885
f-measure,0.326829,0.162206,0.283910


In [12]:
print('Article:',Summaries['Articles'][0])
print('======================================================')
print('Gold Summary:',Summaries['GoldSummary'][0])
print('======================================================')
print('Bart Summary:', Summaries['BartSummary'][0])

Article: The Canadian company Research in Motion Rim, which makes BlackBerry handset, will pruning employees globally in the next few weeks as part of its restructuring scheme. The Globe & Mail said in the news published by the company's nearest sources that Rim has said that Rim is large globally.Preparing restructuring. In the next few weeks, she will cut jobs globally. The report said that the figure of the retrenchment may be even bigger. The Globe & Mail has said in the news published by the company's nearest sources.That Rim is preparing for large restructuring globally. In the next few weeks it will cut jobs globally. The report said that the retrenchment figure can be even bigger than this.
Gold Summary: The Canadian company Research in Motion Rim, which makes BlackBerry handset, will pruning employees globally in the next few weeks under its restructuring plan
Bart Summary: Preparing restructuring. Canadian company Research in Motion Rim, which makes BlackBerry handset, will p

In [13]:
t5_checkpoint = "deep-learning-analytics/wikihow-t5-small"

tokenizer_t5 = AutoTokenizer.from_pretrained(t5_checkpoint)
model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint).to(device)

In [14]:
%%time

SystemSummary1 = []

for i, input_text in enumerate(text):    
    
    input_ids = tokenizer_t5(input_text,
                             return_tensors="pt",
                             padding="max_length",
                             truncation=True,
                             max_length=768)["input_ids"].to(device) 

    model_output = model_t5.generate(input_ids=input_ids,
                                     num_beams=4,
                                     max_length=70,
                                     min_length=30,
                                     no_repeat_ngram_size=2,
                                     early_stopping=True)

    decoded_output = tokenizer_t5.decode(model_output[0],
                                  skip_special_tokens=True,
                                  clean_up_tokenization_spaces=False)

    
    
    SystemSummary1.append(decoded_output)

Wall time: 3min 11s


In [15]:
Summaries['T5Summary'] = SystemSummary1
Summaries.head()

,Articles,GoldSummary,BartSummary,T5Summary
0,"The Canadian company Research in Motion Rim, w...","The Canadian company Research in Motion Rim, w...",Preparing restructuring. Canadian company Rese...,Research in Motion Rim will prune employees gl...
1,"Hillary Clinton, a Democratic Party candidate ...",Hillary Clinton's health that suddenly deterio...,This news has not been edited by the NDTV team...,Read the doctor's statement.Read the NDTV news...
2,"Aditya Ghosh, President and director of Indigo...",Indigo's ground staff has apologized for this ...,"itya Ghosh, President and director of Indigo A...","Aditya Ghosh, President and director of Indigo..."
3,"Mejan Jafri, who made her Bollywood debut with...","Meijan Jafri opted out to Nanda, a good friend...","jan Jafri, who made her Bollywood debut with S...",Talk to the media about Navya Naveli Nanda.Tal...
4,The SIT is being investigated by the Uttar Pra...,The supporters had reached outside the police ...,Uttar Pradesh SIT is being investigated by the...,Investigate the MLA Kuldeep Singh Sengar in th...


In [16]:
rouge = Rouge()
score = rouge.get_scores(Summaries['T5Summary'], Summaries['GoldSummary'], avg=True)
T5Rouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
T5Rouge

,rouge-1,rouge-2,rouge-l
recall,0.284098,0.105003,0.246019
precision,0.264281,0.091502,0.229752
f-measure,0.264988,0.094022,0.230122


In [17]:
# specify the colors you want to use
colors = ['#940000', '#FFFDD0', '#FFCCCB']

fig = px.bar(BartRouge*100, x=BartRouge.index, y=BartRouge.columns, 
             barmode='group', 
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 },color_discrete_sequence=colors)
fig.update_layout( width=650,
                  height=400,
                title={
                  'text': "Bart Rouge Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()

In [20]:

#######################################
import plotly.express as px

# assuming T5Rouge and BartRouge are pandas dataframes
T5Rouge = T5Rouge[['rouge-1', 'rouge-l', 'rouge-2']]
BartRouge = BartRouge[['rouge-1', 'rouge-l', 'rouge-2']]

# specify the colors you want to use
colors = ['#940000', '#FFFDD0', '#FFCCCB']

fig = px.bar(T5Rouge*100, x=T5Rouge.index, y=T5Rouge.columns, 
             barmode='group', 
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 },
             color_discrete_sequence=colors)

fig.update_layout(width=650,
                  height=400,
                  title={
                  'text': "T5 Rougue Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()
